In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import re

In [2]:
import os
# checkpoint directory for stateful operations
checkpoint_dir = '/tmp/spark_streaming_checkpoint'
os.makedirs(checkpoint_dir, exist_ok=True)

def create_streaming_context():
    sc = SparkContext.getOrCreate()
    ssc = StreamingContext(sc, 5)
    ssc.checkpoint(checkpoint_dir)

    # define the stream and processing inside the context creation
    lines = ssc.socketTextStream("localhost", 9999)

    def tokenize(line):
        return re.findall(r"\w+", line.lower())

    words = lines.flatMap(lambda line: tokenize(line))
    pairs = words.map(lambda w: (w, 1))
    wordCount = pairs.reduceByKey(lambda x, y: x + y)

    def update_running_counts(new_values, running_count):
        return sum(new_values) + (running_count or 0)

    runningCounts = pairs.updateStateByKey(update_running_counts)

    wordCount.pprint()
    runningCounts.pprint()

    return ssc

# getOrCreate will reuse existing context if present, avoiding duplicate receivers
ssc = StreamingContext.getOrCreate(checkpoint_dir, create_streaming_context)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/07 23:46:10 WARN Utils: Your hostname, kingstein, resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlp3s0)
25/11/07 23:46:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/07 23:46:10 WARN Utils: Your hostname, kingstein, resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlp3s0)
25/11/07 23:46:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setL

-------------------------------------------
Time: 2025-11-07 23:46:15
-------------------------------------------

-------------------------------------------
Time: 2025-11-07 23:46:15
-------------------------------------------

-------------------------------------------
Time: 2025-11-07 23:46:15
-------------------------------------------

-------------------------------------------
Time: 2025-11-07 23:46:20
-------------------------------------------
('good', 1)

-------------------------------------------
Time: 2025-11-07 23:46:20
-------------------------------------------
('good', 1)

-------------------------------------------
Time: 2025-11-07 23:46:20
-------------------------------------------
('good', 1)

-------------------------------------------
Time: 2025-11-07 23:46:20
-------------------------------------------
('good', 1)

-------------------------------------------
Time: 2025-11-07 23:46:25
-------------------------------------------
('good', 2)

--------------------

In [3]:
ssc.start()

In [ ]:
ssc.awaitTermination()

25/11/07 23:46:19 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/11/07 23:46:19 WARN BlockManager: Block input-0-1762539379400 replicated to only 0 peer(s) instead of 1 peers
25/11/07 23:46:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/11/07 23:46:21 WARN BlockManager: Block input-0-1762539381200 replicated to only 0 peer(s) instead of 1 peers
25/11/07 23:46:24 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/11/07 23:46:24 WARN BlockManager: Block input-0-1762539384600 replicated to only 0 peer(s) instead of 1 peers
25/11/07 23:46:29 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/11/07 23:46:29 WARN BlockManager: Block input-0-1762539389400 replicated to only 0 peer(s) instead of 1 peers
25/11/07 23:46:32 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/11/07 23:46:32 WARN BlockManager: Block input-0-1762539392000 replicated to

KeyboardInterrupt: 

25/11/07 23:47:15 WARN BlockManager: Putting block rdd_306_0 failed due to exception org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/franz/Documents/Lab/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1968, in main
    split_index = read_int(infile)
                  ^^^^^^^^^^^^^^^^
  File "/home/franz/Documents/Lab/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    length = stream.read(4)
             ^^^^^^^^^^^^^^
KeyboardInterrupt
.
25/11/07 23:47:15 WARN BlockManager: Block rdd_306_0 could not be removed as it was not found on disk or in memory
25/11/07 23:47:15 ERROR Executor: Exception in task 0.0 in stage 160.0 (TID 494)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/franz/Documents/Lab/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 19